1\. Write a function that converts number representation (bin<->dec<->hex)

In [62]:
import string

alphabet = list(string.ascii_lowercase)
translator = {i+10:letter for i,letter in enumerate(alphabet)}
de_translator = {value: key for key,value in translator.items()}
codes = {16: 'x', 2: 'b'}

def to_dec(arg,original_base,code):
    nums = reversed(list(arg.split(code)[1]))
    dec = 0
    for i, n in enumerate(nums):
        n = de_translator.get(n,n)
        dec += int(n)*original_base**i
    return dec

def from_dec(arg,new_base,code):
    quo, res = arg//new_base, arg%new_base
    res = translator.get(res,res)
    nums = f'{res}'
    while quo:
        quo, res = quo//new_base, quo%new_base
        res = translator.get(res,res)
        nums = f'{res}'+nums
    num_rep =  '0'+code+nums
    return num_rep 

def convert(arg,original_base,new_base):
    '''Converts an integer number from one base to another.\n
    Processes hexadecimal, decimal and binary.\n
    Returns and expects hexadecimal and binary numbers as strings.'''
    if new_base == original_base:
        # No need to do anything
        return arg
    if new_base == 10:
        return to_dec(arg,original_base,codes[original_base])
    if original_base == 10:
        return from_dec(arg,new_base,codes[new_base])
    # If neither of the above was the case,
    # then it is a conversion from a non decimal base to another.
    # We'll pivot on the decimal base.
    return convert(convert(arg,original_base,10),10,new_base)

    
test_n = 1234
for base, int_base in zip(['hex','bin'],[16,2]):
    print(f"{base}({test_n}): {eval(base+F'({test_n})')} is equal to {convert(test_n,10,int_base)}")
    print(f"convert({eval(base+F'({test_n})')},{int_base},10): {convert(eval(base+F'(test_n)'),int_base,10)} is equal to {test_n}")
    print()


hex(1234): 0x4d2 is equal to 0x4d2
convert(0x4d2,16,10): 1234 is equal to 1234

bin(1234): 0b10011010010 is equal to 0b10011010010
convert(0b10011010010,2,10): 1234 is equal to 1234



2\. Write a function that converts a 32 bit word into a single precision floating point (i.e. interprets the various bits as sign, mantissa and exponent)

In [78]:
!pip install bitstring # Installing bitstring just for result validation
import bitstring
def bin_to_float(arg, n_exp_bits=8, bias=127):
    sign_b = arg[0] # It is assumed that the asign will always be specified by the first bit (from left to right)
    exp_bits = '0b'+arg[1:n_exp_bits+1] # Expressed in this way to easily convert it later on
    mantissa_bits = arg[n_exp_bits+1:] # The rest of bits belong to the mantissa
    number = 1
    _exp = convert(exp_bits,2,10)-bias # Here we use a previously defined function
    for i, b in enumerate(mantissa_bits):
        number+= int(b)*2**(-i-1)
    number*=(-1)**int(sign_b)*2**(_exp)
    return number

test_n = 1.53
bit = bitstring.BitArray(float = test_n, length = 32).bin
print(f'Test number: {test_n}. IEEE754 representation is given by bitstring is {bit}.\n The value of this bitsring is {bin_to_float(bit)}.')

Test number: 1.53. IEEE754 representation is given by bitstring is 00111111110000111101011100001010.
 The value of this bitsring is 1.5299999713897705.


3\. Write a program to determine the underflow and overflow limits (within a factor of 2) for python on your computer. 

**Tips**: define two variables inizialized to 1 and halve/double them enough time to exceed the under/over-flow limits  

In [81]:
a = 1.0
i = 0
r = a*2.0
while r/2.0 == a:# This condition breaks when r no longer accurately represents the integer as a float
    a*=2.0
    r = a*2.0
    i+=1
print(f'This computer has an overflow limit of {2.0**i}')
b = 1.0
j = 0
p = b/2.0
while p*2.0 == b:# This condition breaks when r no longer accurately represents the integer as a float
    b/=2.0
    p = b/2.0
    j+=1
print(f'This computer has an underflow limit of {2**-j}')

This computer has an overflow limit of 8.98846567431158e+307
This computer has an underflow limit of 5e-324


4\. Write a program to determine the machine precision

**Tips**: define a new variable by adding a smaller and smaller value (proceeding similarly to prob. 2) to an original variable and check the point where the two are the same 

In [82]:
target = 1
i = 1
wrong = target + 10**(-i)
while wrong != target:
    i+=1
    wrong = target + 10**(-i)
print(f'Machine precision holds up to 10^-{i-1}')

Machine precision holds up to 10^-15


5\. Write a function that takes in input three parameters $a$, $b$ and $c$ and prints out the two solutions to the quadratic equation $ax^2+bx+c=0$ using the standard formula:
$$
x=\frac{-b\pm\sqrt{b^2-4ac}}{2a}
$$

(a) use the program to compute the solution for $a=0.001$, $b=1000$ and $c=0.001$

(b) re-express the standard solution formula by multiplying top and bottom by $-b\mp\sqrt{b^2-4ac}$ and again find the solution for $a=0.001$, $b=1000$ and $c=0.001$. How does it compare with what previously obtained? Why?

(c) write a function that compute the roots of a quadratic equation accurately in all cases

In [15]:
# a
a,b,c = 0.001,1000,0.001
from math import sqrt, exp, log
det = lambda a,b,c: sqrt(b**2-4*a*c)
gen_quad = lambda a, b, c: ((-b+det(a,b,c))/(2*a), (-b-det(a,b,c))/(2*a))
re_gen_quad = lambda a, b, c: ((-b+det(a,b,c))*(-b-det(a,b,c))/(2*a*(-b-det(a,b,c))), (-b-det(a,b,c))*(-b+det(a,b,c))/(2*a*(-b+det(a,b,c))))
print(gen_quad(a,b,c))
print(re_gen_quad(a,b,c))

def exp_gen_quad(a,b,c):
    print(det(a,b,c))
    print(exp(1000))
    x_1 = exp(-b)*exp(det(a,b,c))
    x_2 = exp(-b)*exp(-det(a,b,c))
    x_1 = log(x_1)
    x_2 = log(x_2)
    return x_1/(2*a), x_2/(2*a)
#def log_gen_quad(a,b,c):
#    x_1 = log(-b+det(a,b,c))-log(2)-log(a)
#    x_2 = log(-b-det(a,b,c))-log(2)-log(a)
#    return exp(x_1), exp(x_2)

print(exp_gen_quad(a,b,c))

(-9.999894245993346e-07, -999999.999999)
(-9.999894245993346e-07, -999999.9999990001)
999.999999998


OverflowError: math range error

6\. Write a program that implements the function $f(x)=x(x−1)$

(a) Calculate the derivative of the function at the point $x = 1$ using the derivative definition:

$$
\frac{{\rm d}f}{{\rm d}x} = \lim_{\delta\to0} \frac{f(x+\delta)-f(x)}{\delta}
$$

with $\delta = 10^{−2}$. Calculate the true value of the same derivative analytically and compare with the answer your program gives. The two will not agree perfectly. Why not?

(b) Repeat the calculation for $\delta = 10^{−4}, 10^{−6}, 10^{−8}, 10^{−10}, 10^{−12}$ and $10^{−14}$. How does the accuracy scales with $\delta$?

In [23]:
f = lambda x: x*(x-1)

def D_x(f,d):
    return lambda x: (f(x+d)-f(x))/d

df = D_x(f,0.0000000001)
print(df(1))
print(2*1-1)

1.000000082840371
1


7\. Consider the integral of the semicircle of radius 1:
$$
I=\int_{-1}^{1} \sqrt(1-x^2) {\rm d}x
$$
which it's known to be $I=\frac{\pi}{2}=1.57079632679...$.
Alternatively we can use the Riemann definition of the integral:
$$
I=\lim_{N\to\infty} \sum_{k=1}^{N} h y_k 
$$

with $h=2/N$ the width of each of the $N$ slices the domain is divided into and where
$y_k$ is the value of the function at the $k-$th slice.

(a) Write a programe to compute the integral with $N=100$. How does the result compares to the true value?

(b) How much can $N$ be increased if the computation needs to be run in less than a second? What is the gain in running it for 1 minute? 


In [22]:
from math import pi
def Int_x(f,N):
    return lambda a,b: sum(f(x/N) for x in range(N*a,N*b))/N
print(f'The actual value is {pi/2}')
print(f'The calculated value is {Int_x(lambda x: sqrt(1-x**2),1000)(-1,1)}')
# Time that shit and set an automatic way to estimate max_N

The actual value is 1.5707963267948966
The calculated value is 1.5707777334555133
